# 금연구역_2차_주소전처리 시도

In [2]:
import numpy as np
import pandas as pd
import re
from tqdm import tqdm

# import tensorflow as tf

import requests
import json


In [3]:
# PATH = 'D:/project/데이터 분석 프로젝트/이노포스트/data'
PATH = 'G:/다른 컴퓨터/My_desktop/project/데이터 분석 프로젝트/이노포스트/data'

df_fail = pd.read_csv(PATH + '/03_금연구역/금연구역_장병용_전처리_실패(11115).csv', encoding ='cp949')
df_save = pd.read_csv(PATH + '/03_금연구역/금연구역_장병용_전처리_성공(32752).csv', encoding ='cp949')

## 실패한 주소 체크

전처리를 시도한 kakao 주소가 아닌 기존의 원주소로 다시 api 호출을 시도해봄

In [4]:
df_fail.shape, df_save.shape

((11115, 10), (32752, 10))

In [5]:
df_fail.head()

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도
0,중구,학교교과교습학원,중앙독서실,목중로26번길,"대전광역시 중구 목중로26번길 , 83 (중촌동, 현대아파트)",83,중구 목중로26번길,NaN,NaN,NaN
1,중구,학교교과교습학원,두드림음악학원,계백로1571번길 46,"대전광역시 중구 계백로1571번길 46 , 2층 (유천동)",2층,중구 계백로1571번길 46,NaN,NaN,NaN
2,중구,어린이운송용 승합자동차,문화어린이집,대전 보문로 207번길14 문화어린이집,"대전 중구 보문로 207번길14 문화어린이집, 문화유치원",문화유치원,중구 대전 보문로 207번길14 문화어린이집,NaN,NaN,NaN
3,중구,어린이운송용 승합자동차,은혜어린이집,대전 태평동 삼부@ 410동 12호,"대전 중구 태평동 삼부@ 410동 12호, 은혜어린이집",은혜어린이집,중구 대전 태평동 삼부@ 410동 12호,NaN,NaN,NaN
4,중구,어린이운송용 승합자동차,꿈나무어린이집,대전 선화로57번길 8,"대전 중구 선화로57번길 8, 4층(선화동) 꿈나무어린이집",4층 꿈나무어린이집,중구 대전 선화로57번길 8,NaN,NaN,NaN


### kakao api 세팅

In [6]:
# # rest키 발급 https://developers.kakao.com/console/app

# 병용 474aa3bdba9bd34f1560a7812fafeb9b
# 재인 08d64ac0652c8f9e1c4ea45118910c53
# 수영 616734aa8214dbf958cfaebd6bd41aec
# 태희 04e7e5d98a2c026956de6095963f28f8

rest_api_key = "616734aa8214dbf958cfaebd6bd41aec"

In [7]:
class KakaoLocalAPI:
    """
    Kakao Local API Controller
    """
    def __init__(self, rest_api_key):
        """
        Rest API key 초기화 및 기능 별 url 설정
        """
        
        # REST API key 설정
        self.rest_api_key = rest_api_key
        self.headers = {"Authorization": "KakaoAK {}".format(rest_api_key)}
        
        # 01 주소검색
        self.URL_01 = "https://dapi.kakao.com/v2/local/search/address.json"
        
    def search_address(self, query, analyze_type = None, page = None, size = None):
        """
        01 주소 검색
        """
        params = {"query": f"{query}"}
    
        if analyze_type != None:
            params["analyze_type"]=f"{analyze_type}"
    
        elif page != None:
            params['page'] = f"{page}"
    
        elif size != None:
            params['size'] = f"{size}"
    
        res = requests.get(self.URL_01, headers = self.headers, params = params)
        document = json.loads(res.text)
    
        return document

In [8]:
kakao = KakaoLocalAPI(rest_api_key)

In [9]:
df_fail2 = df_fail.copy()

In [10]:
add_list = df_fail2['번지주소']
add_list[0]

'대전광역시 중구 목중로26번길  , 83 (중촌동, 현대아파트)'

In [11]:
kakao.search_address(add_list[0])

{'documents': [{'address': {'address_name': '대전 중구 중촌동 116-8',
    'b_code': '3014010400',
    'h_code': '3014056000',
    'main_address_no': '116',
    'mountain_yn': 'N',
    'region_1depth_name': '대전',
    'region_2depth_name': '중구',
    'region_3depth_h_name': '중촌동',
    'region_3depth_name': '중촌동',
    'sub_address_no': '8',
    'x': '127.414436051345',
    'y': '36.3387070586325'},
   'address_name': '대전 중구 목중로26번길 83',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '대전 중구 목중로26번길 83',
    'building_name': '',
    'main_building_no': '83',
    'region_1depth_name': '대전',
    'region_2depth_name': '중구',
    'region_3depth_name': '중촌동',
    'road_name': '목중로26번길',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '127.414436051345',
    'y': '36.3387070586325',
    'zone_no': '34804'},
   'x': '127.414436051345',
   'y': '36.3387070586325'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [12]:
# 호출에 실패한 리스트만 번지주소로 다시 돌려봄
for i in tqdm(range(len(add_list))):
    try :
        df_fail2['동'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['address']['region_3depth_name']
        df_fail2['경도'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['x']
        df_fail2['위도'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['y']
    except:
        pass

print(df_fail2.isnull().sum())

  0%|          | 0/11115 [00:00<?, ?it/s]C:\Users\Helpe\AppData\Local\Temp\ipykernel_15004\3131939199.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fail2['동'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['address']['region_3depth_name']
C:\Users\Helpe\AppData\Local\Temp\ipykernel_15004\3131939199.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fail2['경도'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['x']
C:\Users\Helpe\AppData\Local\Temp\ipykernel_15004\3131939199.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

구               0
구분              0
시설명             0
주소              0
번지주소            0
세부주소        10610
kakao 주소        0
동            6523
위도           6523
경도           6523
dtype: int64


번지주소로 api 호출 후 4592개 성공, 6523개 실패

### 2차 api 호출 성공/실패 데이터프레임 저장

In [23]:
df_2rd_fail= df_fail2[df_fail2['위도'].isnull()]
df_2rd_save = df_fail2[~df_fail2['위도'].isnull()]

print(df_2rd_fail.shape)
print(df_2rd_save.shape)

(6523, 10)
(0, 10)


In [14]:
# 병합
df_save2= pd.concat([df_2rd_save, df_save], 
                    axis=0,
                    join='outer'
                    )


df_save2.reset_index(drop=True)

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도
0,중구,학교교과교습학원,예녹피아노음악종합학원,당디로 45,"대전광역시 중구 당디로 45 , 201호,203호,204호 (산성동, 우성아파트)","201호,203호,204호",중구 당디로 45,산성동,36.308536,127.392653
1,중구,학교교과교습학원,단비종합학원,보문로 180 1층,"대전광역시 중구 보문로 180 1층,2층,3층 (대흥동)","2층,3층",중구 보문로 180 1층,대흥동,36.318974,127.425237
2,중구,학교교과교습학원,국제학원,대둔산로 437,"대전광역시 중구 대둔산로 437 , 3층 (산성동)",3층,중구 대둔산로 437,산성동,36.308373,127.387239
3,중구,학교교과교습학원,두루음악학원,당디로6번길 76,"대전광역시 중구 당디로6번길 76 , 403호 (문화동)",403호,중구 당디로6번길 76,문화동,36.307305,127.398404
4,중구,학교교과교습학원,예림음악학원,계백로1615번길 47,"대전광역시 중구 계백로1615번길 47 , 3층 (유천동)",3층,중구 계백로1615번길 47,유천동,36.321016,127.398592
...,...,...,...,...,...,...,...,...,...,...
32747,유성구,음식점,순카페,관평동 666번지 테크노벨리4단지상가 101호 102호,대전광역시 유성구 관평동 666번지 테크노벨리4단지상가 101호 102호,NaN,유성구 관평동 666번지 테크노벨리4단지상가 101호 102호,관평동,36.41951,127.384518
32748,유성구,음식점,선비꼬마김밥 관평점,관평동 668번지 대덕테크노밸리3단지아파트상가동 101-1호,대전광역시 유성구 관평동 668번지 대덕테크노밸리3단지아파트상가동 101-1호,NaN,유성구 관평동 668번지 대덕테크노밸리3단지아파트상가동 101-1호,관평동,36.421222,127.385636
32749,유성구,음식점,김밥도둑,관평동 669번지 대덕테크노밸리5단지 분산상가동 104호,대전광역시 유성구 관평동 669번지 대덕테크노밸리5단지(아)분산상가동 104호,NaN,유성구 관평동 669번지 대덕테크노밸리5단지 분산상가동 104호,관평동,36.420075,127.387263
32750,유성구,음식점,가마치통닭 관평점,관평동 669번지 분산상가동 지상1층 101호,대전광역시 유성구 관평동 669번지 분산상가동 지상1층 101호,NaN,유성구 관평동 669번지 분산상가동 지상1층 101호,관평동,36.420075,127.387263


In [ ]:
# csv로 저장
#df_save2.to_csv(PATH + "/03_금연구역/금연구역_장병용_전처리_성공(35537).csv", index=False, encoding='cp949')
#df_fail2.to_csv(PATH + "/03_금연구역/금연구역_장병용_전처리_실패(6288).csv", index=False, encoding='cp949')

## 무엇이 문제일까 - 실패한 주소지 탐색

In [72]:
df_fail4 = df_fail3.copy()
df_fail4.shape

(6523, 10)

### 주소 컬럼에서 대전 검색

In [73]:
# 주소에서 대전 검색

df_fail4[df_fail4['주소'].str.contains('대전')]

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도
2,중구,어린이운송용 승합자동차,문화어린이집,대전 보문로 207번길14 문화어린이집,"대전 중구 보문로 207번길14 문화어린이집, 문화유치원",문화유치원,중구 대전 보문로 207번길14 문화어린이집,NaN,NaN,NaN
3,중구,어린이운송용 승합자동차,은혜어린이집,대전 태평동 삼부@ 410동 12호,"대전 중구 태평동 삼부@ 410동 12호, 은혜어린이집",은혜어린이집,중구 대전 태평동 삼부@ 410동 12호,NaN,NaN,NaN
4,중구,어린이운송용 승합자동차,꿈나무어린이집,대전 선화로57번길 8,"대전 중구 선화로57번길 8, 4층(선화동) 꿈나무어린이집",4층 꿈나무어린이집,중구 대전 선화로57번길 8,NaN,NaN,NaN
27,중구,"음식점(휴게, 일반, 제과)",우정집실내포장마차앤신존떡볶이,대전 천서로 583,"대전광역시 중구 대전천서로 583, 1층 (선화동)",1층,중구 대전 천서로 583,NaN,NaN,NaN
87,동구,음식점,미자쌀롱우송대점,동대전 로 192-1,"대전광역시 동구 동대전로 192-1, 1층 (자양동)",1층,동구 동대전 로 192-1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2959,유성구,음식점,청춘아 달리자,도룡동 3번지 8호 대전 무역전시관,대전광역시 유성구 도룡동 3번지 8호 대전무역전시관,NaN,유성구 도룡동 3번지 8호 대전 무역전시관,NaN,NaN,NaN
3280,유성구,음식점,보람있는집,방현동 0번지 북대전 IC 졸음쉼터,대전광역시 유성구 방현동 0번지 북대전IC(논산)졸음쉼터,NaN,유성구 방현동 0번지 북대전 IC 졸음쉼터,NaN,NaN,NaN
7094,유성구,담배소매업소,조은마트,노은동 510번지 7호 대전 회관주차장빌딩.라은빌딩,대전광역시 유성구 노은동 510번지 7호 대전회관주차장빌딩.라은빌딩,NaN,유성구 노은동 510번지 7호 대전 회관주차장빌딩.라은빌딩,NaN,NaN,NaN
7120,유성구,담배소매업소,신혜자,덕명동 산16번지 1호 대전 산업대 잡화코너,대전광역시 유성구 덕명동 산16번지 1호 대전산업대 잡화코너 (3층),NaN,유성구 덕명동 산16번지 1호 대전 산업대 잡화코너,NaN,NaN,NaN


In [76]:
# 주소에서 대전 삭제
df_fail4["주소"] = df_fail4["주소"].str.replace(pat='대전', repl=r' ', regex=True)
df_fail4[df_fail4['주소'].str.contains('대전')]

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도


#### 주소에서 대전 제거 후 중복된 공백제거

In [77]:
print(df_fail4['주소'].str.contains('  ').sum()) # 스페이스바 두번
print(df_fail4['주소'].str.contains('   ').sum()) # 스페이스바 세번
print(df_fail4['주소'].str.contains('    ').sum()) # 스페이스바 네번
print(df_fail4['주소'].str.contains('     ').sum()) # 스페이스바 다섯번

74
11
2
0


In [79]:
# 공백 제거
df_fail4["주소"] = df_fail4["주소"].str.replace(pat='  ', repl=r' ', regex=True)
print(df_fail4['주소'].str.contains('  ').sum()) # 스페이스바 두번
print(df_fail4['주소'].str.contains('   ').sum()) # 스페이스바 세번
print(df_fail4['주소'].str.contains('    ').sum()) # 스페이스바 네번
print(df_fail4['주소'].str.contains('     ').sum()) # 스페이스바 다섯번

0
0
0
0


In [80]:
df_dong = df_fail4.copy()
df_dong.reset_index(drop=True, inplace=True)
df_dong.shape

(6523, 10)

### 번지주소 형태 체크
`**동 + 지번` 형태로 구성된 번지주소  
    * 동 뒤에 공백 삽입
    * 중복 공백 제거

In [81]:
df_dong[df_dong['주소'].str.contains('동')]

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도
2,중구,어린이운송용 승합자동차,은혜어린이집,태평동 삼부@ 410동 12호,"대전 중구 태평동 삼부@ 410동 12호, 은혜어린이집",은혜어린이집,중구 대전 태평동 삼부@ 410동 12호,NaN,NaN,NaN
10,중구,"음식점(휴게, 일반, 제과)",맥주마을치킨,동서대로1466번길 25,"대전광역시 중구 동서대로1466번길 25, 1층 (선화동)",1층,중구 동서대로1466번길 25,NaN,NaN,NaN
13,중구,"음식점(휴게, 일반, 제과)",멕시카나치킨목동점,목동로22번길 37,"대전광역시 중구 목동로22번길 37, 1층 (목동)",1층,중구 목동로22번길 37,NaN,NaN,NaN
16,중구,"음식점(휴게, 일반, 제과)",카와17-15브런치,목동로22번길 29,"대전광역시 중구 목동로22번길 29, 1층 (목동)",1층,중구 목동로22번길 29,NaN,NaN,NaN
17,중구,"음식점(휴게, 일반, 제과)",자매식당,동서대로1466번길 18,"대전광역시 중구 동서대로1466번길 18, 1층 (선화동)",1층,중구 동서대로1466번길 18,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
6518,유성구,복합용도,대전서남부 집단에너지시설,원신흥동100-0,대전광역시 유성구 원신흥동100-0,NaN,유성구 원신흥동100-0,NaN,NaN,NaN
6519,유성구,복합용도,대전서남부 집단에너지시설,원신흥동100-0,대전광역시 유성구 원신흥동100-0,NaN,유성구 원신흥동100-0,NaN,NaN,NaN
6520,유성구,복합용도,대전서남부 집단에너지시설,원신흥동100-0,대전광역시 유성구 원신흥동100-0,NaN,유성구 원신흥동100-0,NaN,NaN,NaN
6521,유성구,복합용도,정보통신학교 교육생 숙소,자운동223-0,대전광역시 유성구 자운동223-0,NaN,유성구 자운동223-0,NaN,NaN,NaN


6523개 중 6243개가 동이 들어감
1) 번지주소인 경우
2) 도로명주소지만 도로명에 동이 들어가는 경우

In [82]:
# 동 뒤에 공백 삽입
df_dong['주소'] = df_dong['주소'].str.replace('동', '동 ')

In [83]:
print(df_dong['주소'].str.contains('  ').sum()) # 스페이스바 두번
print(df_dong['주소'].str.contains('   ').sum()) # 스페이스바 세번

6177
0


In [84]:
# 중복 공백 제거
df_dong["주소"] = df_dong["주소"].str.replace(pat='  ', repl=r' ', regex=True)
print(df_dong['주소'].str.contains('  ').sum()) # 스페이스바 두번

0


In [89]:
# 맨 앞에 공백 제거
df_dong["주소"] = df_dong["주소"].str.lstrip()

In [90]:
df_dong.reset_index(drop=True, inplace=True)
df_dong

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도
0,중구,학교교과교습학원,두드림음악학원,계백로1571번길 46,"대전광역시 중구 계백로1571번길 46 , 2층 (유천동)",2층,중구 계백로1571번길 46,NaN,NaN,NaN
1,중구,어린이운송용 승합자동차,문화어린이집,보문로 207번길14 문화어린이집,"대전 중구 보문로 207번길14 문화어린이집, 문화유치원",문화유치원,중구 대전 보문로 207번길14 문화어린이집,NaN,NaN,NaN
2,중구,어린이운송용 승합자동차,은혜어린이집,태평동 삼부@ 410동 12호,"대전 중구 태평동 삼부@ 410동 12호, 은혜어린이집",은혜어린이집,중구 대전 태평동 삼부@ 410동 12호,NaN,NaN,NaN
3,중구,어린이운송용 승합자동차,꿈나무어린이집,선화로57번길 8,"대전 중구 선화로57번길 8, 4층(선화동) 꿈나무어린이집",4층 꿈나무어린이집,중구 대전 선화로57번길 8,NaN,NaN,NaN
4,중구,어린이운송용 승합자동차,산성검도관,보문로55,"대전시 중구 보문로55 ,(산성검도관)",,중구 보문로55,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
6518,유성구,복합용도,대전서남부 집단에너지시설,원신흥동 100-0,대전광역시 유성구 원신흥동100-0,NaN,유성구 원신흥동100-0,NaN,NaN,NaN
6519,유성구,복합용도,대전서남부 집단에너지시설,원신흥동 100-0,대전광역시 유성구 원신흥동100-0,NaN,유성구 원신흥동100-0,NaN,NaN,NaN
6520,유성구,복합용도,대전서남부 집단에너지시설,원신흥동 100-0,대전광역시 유성구 원신흥동100-0,NaN,유성구 원신흥동100-0,NaN,NaN,NaN
6521,유성구,복합용도,정보통신학교 교육생 숙소,자운동 223-0,대전광역시 유성구 자운동223-0,NaN,유성구 자운동223-0,NaN,NaN,NaN


동부로, 중부로 등 도로명 주소 사이에 공백이 삽입된 케이스 존재

In [97]:
df_dong['주소'].str[:3].unique()

array(['계백로', '보문로', '태평동', '선화로', '대종로', '목중로', '계룡로', '동서대', '중앙로',
       '목동로', '용두로', '어덕마', '천서로', '보문산', '대둔산', '중교로', '동성대', '비래서',
       '계족로', '대화로', '한밭대', '대청로', '대덕대', '엣신탄', '동춘당', '석봉로', '도산로',
       '둔산로', '둔산중', '문정로', '오량2', '정림로', '용문로', '만년동', '가수원', '둔산동',
       '고니길', '천동초', '효천4', '동로 ', '효천1', '우암로', '새둑길', '충정로', '천동로',
       '갑동 ', '계산동', '관평동', '구암동', '궁동 ', '노은동', '대정동', '덕명동', '학하서',
       '덕진동', '도룡동', '문지동', '반석동', '방현동', '복용동', '봉명동', '상대동', '송강동',
       '신성동', '용계동', '원내동', '원신흥', '원촌동', '자운동', '장대동', '장동 ', '전민동',
       '죽동 ', '지족동', '자운로', '추목동', '탑립동', '하기동', '학하동', '봉산로', '송강로',
       '신성남', '유성대', '은구비', '상대남', '전민로', '목동 ', '문화동', '부사동', '유천2',
       '산성동', '문화 ', '유천동', '옥계동', '호동 ', '중촌동', '유천로', '동서로', '문화1',
       '학고개', '문로호', '용두동', '문화로', '유등천', '서문로', '대흥로', '선화동', '산성로',
       '문창동', '태평로', '송촌동', '신탄진', '오정동', '석봉동', '법동 ', '갑천도', '비공개',
       '상서동', '중리동', '목상동', '읍내동', '중리남', '남경마', '문평로', '송촌로', '계족산',
       '덕암로', '다사랑',

In [94]:

df_dong['주소'] = df_dong['주소'].str.replace('동 부', '동부')
df_dong['주소'] = df_dong['주소'].str.replace('동 춘', '동춘')
df_dong['주소'] = df_dong['주소'].str.replace('동 서', '동서')
df_dong['주소'] = df_dong['주소'].str.replace('천동 초', '천동초')
df_dong['주소'] = df_dong['주소'].str.replace('동 성', '동성')
df_dong['주소'] = df_dong['주소'].str.replace('동 대', '동대')
df_dong['주소'] = df_dong['주소'].str.replace('동 로', '동로')

In [96]:
df_dong['주소'].str[:3].unique()

array(['계백로', '보문로', '태평동', '선화로', '대종로', '목중로', '계룡로', '동서대', '중앙로',
       '목동로', '용두로', '어덕마', '천서로', '보문산', '대둔산', '중교로', '동성대', '비래서',
       '계족로', '대화로', '한밭대', '대청로', '대덕대', '엣신탄', '동춘당', '석봉로', '도산로',
       '둔산로', '둔산중', '문정로', '오량2', '정림로', '용문로', '만년동', '가수원', '둔산동',
       '고니길', '천동초', '효천4', '동로 ', '효천1', '우암로', '새둑길', '충정로', '천동로',
       '갑동 ', '계산동', '관평동', '구암동', '궁동 ', '노은동', '대정동', '덕명동', '학하서',
       '덕진동', '도룡동', '문지동', '반석동', '방현동', '복용동', '봉명동', '상대동', '송강동',
       '신성동', '용계동', '원내동', '원신흥', '원촌동', '자운동', '장대동', '장동 ', '전민동',
       '죽동 ', '지족동', '자운로', '추목동', '탑립동', '하기동', '학하동', '봉산로', '송강로',
       '신성남', '유성대', '은구비', '상대남', '전민로', '목동 ', '문화동', '부사동', '유천2',
       '산성동', '문화 ', '유천동', '옥계동', '호동 ', '중촌동', '유천로', '동서로', '문화1',
       '학고개', '문로호', '용두동', '문화로', '유등천', '서문로', '대흥로', '선화동', '산성로',
       '문창동', '태평로', '송촌동', '신탄진', '오정동', '석봉동', '법동 ', '갑천도', '비공개',
       '상서동', '중리동', '목상동', '읍내동', '중리남', '남경마', '문평로', '송촌로', '계족산',
       '덕암로', '다사랑',

#### 구 + 주소 결합

In [98]:
df_dong['kakao 주소'] = df_dong['구'].str.cat(df_dong['주소'], sep=' ')
df_dong

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도
0,중구,학교교과교습학원,두드림음악학원,계백로1571번길 46,"대전광역시 중구 계백로1571번길 46 , 2층 (유천동)",2층,중구 계백로1571번길 46,NaN,NaN,NaN
1,중구,어린이운송용 승합자동차,문화어린이집,보문로 207번길14 문화어린이집,"대전 중구 보문로 207번길14 문화어린이집, 문화유치원",문화유치원,중구 보문로 207번길14 문화어린이집,NaN,NaN,NaN
2,중구,어린이운송용 승합자동차,은혜어린이집,태평동 삼부@ 410동 12호,"대전 중구 태평동 삼부@ 410동 12호, 은혜어린이집",은혜어린이집,중구 태평동 삼부@ 410동 12호,NaN,NaN,NaN
3,중구,어린이운송용 승합자동차,꿈나무어린이집,선화로57번길 8,"대전 중구 선화로57번길 8, 4층(선화동) 꿈나무어린이집",4층 꿈나무어린이집,중구 선화로57번길 8,NaN,NaN,NaN
4,중구,어린이운송용 승합자동차,산성검도관,보문로55,"대전시 중구 보문로55 ,(산성검도관)",,중구 보문로55,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
6518,유성구,복합용도,대전서남부 집단에너지시설,원신흥동 100-0,대전광역시 유성구 원신흥동100-0,NaN,유성구 원신흥동 100-0,NaN,NaN,NaN
6519,유성구,복합용도,대전서남부 집단에너지시설,원신흥동 100-0,대전광역시 유성구 원신흥동100-0,NaN,유성구 원신흥동 100-0,NaN,NaN,NaN
6520,유성구,복합용도,대전서남부 집단에너지시설,원신흥동 100-0,대전광역시 유성구 원신흥동100-0,NaN,유성구 원신흥동 100-0,NaN,NaN,NaN
6521,유성구,복합용도,정보통신학교 교육생 숙소,자운동 223-0,대전광역시 유성구 자운동223-0,NaN,유성구 자운동 223-0,NaN,NaN,NaN


In [103]:
# @ 제거
df_dong['kakao 주소'] = df_dong['kakao 주소'].str.replace('@', ' ')
df_dong[df_dong['주소'].str.contains('@')]

### 3차 api 호출

In [106]:
df_3rd = df_dong.copy()

In [109]:
road_list = df_3rd['kakao 주소']
kakao.search_address(road_list[2])

{'documents': [],
 'meta': {'is_end': True, 'pageable_count': 0, 'total_count': 0}}

In [108]:
for i in tqdm(range(len(road_list))):
    try :
        df_3rd['동'].iloc[i] = kakao.search_address(road_list[i])['documents'][0]['address']['region_3depth_name']
        df_3rd['경도'].iloc[i] = kakao.search_address(road_list[i])['documents'][0]['x']
        df_3rd['위도'].iloc[i] = kakao.search_address(road_list[i])['documents'][0]['y']
    except:
        pass

print(df_3rd.isnull().sum())

100%|██████████| 6523/6523 [09:00<00:00, 12.06it/s]


구              0
구분             0
시설명            0
주소             0
번지주소           0
세부주소        6043
kakao 주소       0
동           6523
위도          6523
경도          6523
dtype: int64


3차시도로 0개 추출에 성공

### 대전 + 구 + 시설명으로 검색 시도

In [132]:
b = '대전'
df_djgu = df_3rd.copy()
df_djgu['광역시'] = b
df_djgu.head()

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도,광역시
0,중구,학교교과교습학원,두드림음악학원,계백로1571번길 46,"대전광역시 중구 계백로1571번길 46 , 2층 (유천동)",2층,중구 계백로1571번길 46,NaN,NaN,NaN,대전
1,중구,어린이운송용 승합자동차,문화어린이집,보문로 207번길14 문화어린이집,"대전 중구 보문로 207번길14 문화어린이집, 문화유치원",문화유치원,중구 보문로 207번길14 문화어린이집,NaN,NaN,NaN,대전
2,중구,어린이운송용 승합자동차,은혜어린이집,태평동 삼부@ 410동 12호,"대전 중구 태평동 삼부@ 410동 12호, 은혜어린이집",은혜어린이집,중구 태평동 삼부 410동 12호,NaN,NaN,NaN,대전
3,중구,어린이운송용 승합자동차,꿈나무어린이집,선화로57번길 8,"대전 중구 선화로57번길 8, 4층(선화동) 꿈나무어린이집",4층 꿈나무어린이집,중구 선화로57번길 8,NaN,NaN,NaN,대전
4,중구,어린이운송용 승합자동차,산성검도관,보문로55,"대전시 중구 보문로55 ,(산성검도관)",,중구 보문로55,NaN,NaN,NaN,대전


In [133]:
df_djgu['kakao 주소'] = df_djgu['광역시'].str.cat(df_djgu['구'], sep=' ')
df_djgu['kakao 주소'] = df_djgu['kakao 주소'].str.cat(df_djgu['시설명'], sep=' ')
df_djgu

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도,광역시
0,중구,학교교과교습학원,두드림음악학원,계백로1571번길 46,"대전광역시 중구 계백로1571번길 46 , 2층 (유천동)",2층,대전 중구 두드림음악학원,NaN,NaN,NaN,대전
1,중구,어린이운송용 승합자동차,문화어린이집,보문로 207번길14 문화어린이집,"대전 중구 보문로 207번길14 문화어린이집, 문화유치원",문화유치원,대전 중구 문화어린이집,NaN,NaN,NaN,대전
2,중구,어린이운송용 승합자동차,은혜어린이집,태평동 삼부@ 410동 12호,"대전 중구 태평동 삼부@ 410동 12호, 은혜어린이집",은혜어린이집,대전 중구 은혜어린이집,NaN,NaN,NaN,대전
3,중구,어린이운송용 승합자동차,꿈나무어린이집,선화로57번길 8,"대전 중구 선화로57번길 8, 4층(선화동) 꿈나무어린이집",4층 꿈나무어린이집,대전 중구 꿈나무어린이집,NaN,NaN,NaN,대전
4,중구,어린이운송용 승합자동차,산성검도관,보문로55,"대전시 중구 보문로55 ,(산성검도관)",,대전 중구 산성검도관,NaN,NaN,NaN,대전
...,...,...,...,...,...,...,...,...,...,...,...
6518,유성구,복합용도,대전서남부 집단에너지시설,원신흥동 100-0,대전광역시 유성구 원신흥동100-0,NaN,대전 유성구 대전서남부 집단에너지시설,NaN,NaN,NaN,대전
6519,유성구,복합용도,대전서남부 집단에너지시설,원신흥동 100-0,대전광역시 유성구 원신흥동100-0,NaN,대전 유성구 대전서남부 집단에너지시설,NaN,NaN,NaN,대전
6520,유성구,복합용도,대전서남부 집단에너지시설,원신흥동 100-0,대전광역시 유성구 원신흥동100-0,NaN,대전 유성구 대전서남부 집단에너지시설,NaN,NaN,NaN,대전
6521,유성구,복합용도,정보통신학교 교육생 숙소,자운동 223-0,대전광역시 유성구 자운동223-0,NaN,대전 유성구 정보통신학교 교육생 숙소,NaN,NaN,NaN,대전


### kakao api 호출

In [134]:
df_4th = df_djgu.copy()

In [139]:
name_list = df_4th['kakao 주소']
kakao.search_address('대전 중구 두드림음악학원')

{'documents': [],
 'meta': {'is_end': True, 'pageable_count': 0, 'total_count': 0}}

In [136]:
for i in tqdm(range(len(road_list))):
    try :
        df_4th['동'].iloc[i] = kakao.search_address(road_list[i])['documents'][0]['address']['region_3depth_name']
        df_4th['경도'].iloc[i] = kakao.search_address(road_list[i])['documents'][0]['x']
        df_4th['위도'].iloc[i] = kakao.search_address(road_list[i])['documents'][0]['y']
    except:
        pass

print(df_4th.isnull().sum())

100%|██████████| 6523/6523 [08:50<00:00, 12.30it/s]

구              0
구분             0
시설명            0
주소             0
번지주소           0
세부주소        6043
kakao 주소       0
동           6523
위도          6523
경도          6523
광역시            0
dtype: int64


In [ ]:
df_3rd_save = df_3rd[df_3rd['위도'].isnull()]
df_3rd_fail = df_3rd[~df_3rd['위도'].isnull()]

print(df_3rd_fail.shape)
print(df_3rd_save.shape)

In [ ]:
# 병합
df_save2= pd.concat([df_save_dong, df_save], 
                    axis=0,
                    join='outer'
                    )


df_save2.reset_index(drop=True)

## 상권정보에서 상호명으로 검색

In [153]:
df_nsmo = df_4th.copy()
df_nsmo.shape

(6523, 11)

In [157]:
df_nsmo['시설명'].nunique()

6250

In [140]:
PATH2 = 'G:/다른 컴퓨터/My_desktop/project/데이터 분석 프로젝트/이노포스트/data'

shop = pd.read_csv(PATH2 + '/07_상권정보/소상공인시장진흥공단_상가(상권)정보_대전_202209.csv')

In [141]:
shop.columns

Index(['상가업소번호', '상호명', '지점명', '상권업종대분류코드', '상권업종대분류명', '상권업종중분류코드',
       '상권업종중분류명', '상권업종소분류코드', '상권업종소분류명', '표준산업분류코드', '표준산업분류명', '시도코드',
       '시도명', '시군구코드', '시군구명', '행정동코드', '행정동명', '법정동코드', '법정동명', '지번코드',
       '대지구분코드', '대지구분명', '지번본번지', '지번부번지', '지번주소', '도로명코드', '도로명', '건물본번지',
       '건물부번지', '건물관리번호', '건물명', '도로명주소', '구우편번호', '신우편번호', '동정보', '층정보',
       '호정보', '경도', '위도'],
      dtype='object')

In [145]:
# 1차 삭제
shop_drop = shop.drop(['상가업소번호',
                   '지점명',
                   '상권업종대분류코드',
                   '상권업종중분류코드',
                   '표준산업분류코드',
                   '상권업종소분류코드',
                   '시도코드',
                   '시군구코드',
                   '행정동코드',
                   '법정동코드',
                   '지번코드',
                   '지번본번지',
                   '지번부번지',
                   '지번주소',
                   '구우편번호'
                   ,'신우편번호'
                   ,'대지구분코드',
                   '대지구분명'
                   ,'도로명코드'
                   ,'건물부번지'
                   ,'건물관리번호'
                   ,'동정보'
                   ,'층정보'
                   ,'호정보'
                   ],
                  axis ='columns')

shop_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75145 entries, 0 to 75144
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   상호명       75145 non-null  object 
 1   상권업종대분류명  75145 non-null  object 
 2   상권업종중분류명  75145 non-null  object 
 3   상권업종소분류명  75145 non-null  object 
 4   표준산업분류명   69614 non-null  object 
 5   시도명       75145 non-null  object 
 6   시군구명      75145 non-null  object 
 7   행정동명      75145 non-null  object 
 8   법정동명      75145 non-null  object 
 9   도로명       75145 non-null  object 
 10  건물본번지     75145 non-null  int64  
 11  건물명       23500 non-null  object 
 12  도로명주소     75145 non-null  object 
 13  경도        75145 non-null  float64
 14  위도        75145 non-null  float64
dtypes: float64(2), int64(1), object(12)
memory usage: 8.6+ MB


In [146]:
# 2차 삭제
shop_drop2 = shop_drop.drop(['상권업종대분류명'
                            ,'상권업종중분류명'
                            ,'상권업종소분류명'
                            ,'표준산업분류명'
                            ,'건물본번지'
                            ,'건물명'
                            ], axis ='columns')
shop_drop2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75145 entries, 0 to 75144
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   상호명     75145 non-null  object 
 1   시도명     75145 non-null  object 
 2   시군구명    75145 non-null  object 
 3   행정동명    75145 non-null  object 
 4   법정동명    75145 non-null  object 
 5   도로명     75145 non-null  object 
 6   도로명주소   75145 non-null  object 
 7   경도      75145 non-null  float64
 8   위도      75145 non-null  float64
dtypes: float64(2), object(7)
memory usage: 5.2+ MB


In [160]:
shop_drop2[shop_drop2['상호명'].str.contains('은혜어린이집')]

,상호명,시도명,시군구명,행정동명,법정동명,도로명,도로명주소,경도,위도
12334,은혜어린이집,대전광역시,서구,정림동,정림동,대전광역시 서구 정림서로,"대전광역시 서구 정림서로 162-18, (정림동, 강변들보람아파트)",127.364056,36.301195


In [161]:
df_nsmo[df_nsmo['시설명'].str.contains('은혜어린이집')]

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도,광역시
2,중구,어린이운송용 승합자동차,은혜어린이집,태평동 삼부@ 410동 12호,"대전 중구 태평동 삼부@ 410동 12호, 은혜어린이집",은혜어린이집,대전 중구 은혜어린이집,NaN,NaN,NaN,대전


In [158]:
df_nsmo.columns

Index(['구', '구분', '시설명', '주소', '번지주소', '세부주소', 'kakao 주소', '동', '위도', '경도',
       '광역시'],
      dtype='object')

In [ ]:
df_merge = pd.merge()

# 